In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

import env

## acquire & summarize
1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [4]:
url = env.get_db_url("zillow")
query= '''
    SELECT prop.*,
	pred.logerror,
	pred.transactiondate,	
	cons.typeconstructiondesc,
	air.airconditioningdesc,
	arch.architecturalstyledesc,
	build.buildingclassdesc,
	land.propertylandusedesc,		
	story.storydesc
	
FROM properties_2017 prop
	INNER JOIN(SELECT parcelid, logerror, MAX(transactiondate)transactiondate
				  FROM predictions_2017
				  GROUP BY parcelid, logerror) pred
			USING (parcelid)
	LEFT JOIN typeconstructiontype cons USING (typeconstructiontypeid)
	LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
	LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
	LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
	LEFT JOIN propertylandusetype land USING (propertylandusetypeid)
	LEFT JOIN storytype story USING (storytypeid)
WHERE prop.latitude IS NOT NULL
	AND prop.longitude IS NOT NULL
	AND transactiondate LIKE '2017%%';
'''


In [5]:
df = pd.read_sql(query, url)

In [6]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxdelinquencyyear,censustractandblock,logerror,transactiondate,typeconstructiondesc,airconditioningdesc,architecturalstyledesc,buildingclassdesc,propertylandusedesc,storydesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,NaN,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,NaN,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,NaN,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,NaN,6.037300e+13,-0.103410,2017-01-01,None,None,None,None,Single Family Residential,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,NaN,6.037124e+13,0.006940,2017-01-01,None,Central,None,None,Condominium,None


In [9]:
df.to_csv("zillow.csv")

#### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77574 entries, 0 to 77573
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77574 non-null  int64  
 1   parcelid                      77574 non-null  int64  
 2   airconditioningtypeid         25006 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77574 non-null  float64
 6   bedroomcnt                    77574 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49808 non-null  float64
 9   calculatedbathnbr             76959 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6035 non-null   float64
 12  calculatedfinishedsquarefeet  77374 non-null  float64
 13  f

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,77574.0,1.495340e+06,8.609689e+05,3.490000e+02,7.520875e+05,1.498169e+06,2.240588e+06,2.982274e+06
parcelid,77574.0,1.300630e+07,3.478043e+06,1.071186e+07,1.153819e+07,1.253050e+07,1.421124e+07,1.676893e+08
airconditioningtypeid,25006.0,1.812045e+00,2.965823e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03
bathroomcnt,77574.0,2.298521e+00,9.967057e-01,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
bedroomcnt,77574.0,3.053252e+00,1.140447e+00,0.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,1.600000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
buildingqualitytypeid,49808.0,6.533830e+00,1.722041e+00,1.000000e+00,6.000000e+00,6.000000e+00,8.000000e+00,1.200000e+01
calculatedbathnbr,76959.0,2.316389e+00,9.796902e-01,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01


In [10]:
df.shape

(77574, 67)

In [14]:
df.isnull().sum()

id                              0
parcelid                        0
airconditioningtypeid       52568
architecturalstyletypeid    77368
basementsqft                77524
                            ...  
airconditioningdesc         52568
architecturalstyledesc      77368
buildingclassdesc           77559
propertylandusedesc             0
storydesc                   77524
Length: 67, dtype: int64

##### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


||num_rows_missing	|pct_rows_missing|
|---|---|---|
|parcelid|	0	|0.000000|
|airconditioningtypeid|	29041 |	0.535486|
|architecturalstyletypeid|	54232 |	0.999982|


In [19]:
num_rows_missing = df.isnull().sum()
df_null

id                              0
parcelid                        0
airconditioningtypeid       52568
architecturalstyletypeid    77368
basementsqft                77524
                            ...  
airconditioningdesc         52568
architecturalstyledesc      77368
buildingclassdesc           77559
propertylandusedesc             0
storydesc                   77524
Length: 67, dtype: int64

In [20]:
num_rows = df.shape[0]
num_rows

77574

In [22]:
# Calculate percent 
pct_rows_missing = num_rows_missing/num_rows
pct_rows_missing

id                          0.000000
parcelid                    0.000000
airconditioningtypeid       0.677650
architecturalstyletypeid    0.997344
basementsqft                0.999355
                              ...   
airconditioningdesc         0.677650
architecturalstyledesc      0.997344
buildingclassdesc           0.999807
propertylandusedesc         0.000000
storydesc                   0.999355
Length: 67, dtype: float64

In [23]:
# Make dataframe
pd.DataFrame({'number_missing_rows': num_rows_missing, 'percent_rows_missing': pct_rows_missing})

,number_missing_rows,percent_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52568,0.677650
architecturalstyletypeid,77368,0.997344
basementsqft,77524,0.999355
...,...,...
airconditioningdesc,52568,0.677650
architecturalstyledesc,77368,0.997344
buildingclassdesc,77559,0.999807
propertylandusedesc,0,0.000000


In [24]:
def nulls_by_col(df):
    num_rows_missing = df.isnull().sum()
    pct_rows_missing = num_rows_missing/num_rows
    df_missing = pd.DataFrame({'number_missing_rows': num_rows_missing, 'percent_rows_missing': pct_rows_missing})
    return df_missing
    

In [26]:
nulls_by_col(df)

,number_missing_rows,percent_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52568,0.677650
architecturalstyletypeid,77368,0.997344
basementsqft,77524,0.999355
...,...,...
airconditioningdesc,52568,0.677650
architecturalstyledesc,77368,0.997344
buildingclassdesc,77559,0.999807
propertylandusedesc,0,0.000000


## Prepare

 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

 - The input:
      - A datafram:
      - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing):
      - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

 - The output:
      - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

 - hint:
      - Look up the dropna documentation:
      - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns:
      - Make use of inplace, i.e. inplace=True/False.

 3. Decide how to handle the remaining missing values:

 - Fill with constant value.
 - Impute with mean, median, mode.
 - Drop row/column

### wrangle_zillow.py
Functions of the work above needed to acquire and prepare a new sample of data.




## Mall Customers

### notebook
1. Acquire data from mall_customers.customers in mysql database.
2. Summarize data (include distributions and descriptive statistics).
3. Detect outliers using IQR.
4. Split data (train, validate, and test split).
5. Encode categorical columns using a one hot encoder (pd.get_dummies).
6. Handles missing values.
7. Scaling

### wrangle_mall.py
1. Acquire data from mall_customers.customers in mysql database.
2. Split the data into train, validate, and split
3. One-hot-encoding (pd.get_dummies)
4. Missing values
5. Scaling
Add, commit, and push all of your work.
